In [1]:
import pickle
import numpy as np

# Replace 'your_file.pkl' with the path to your pickle file
with open('/home/nitesh/workspace/offline_rl_test/replay_buffer_7.pkl', 'rb') as f:
    data = pickle.load(f)


In [2]:
import pandas as pd

df = pd.DataFrame(data)

In [3]:
df.head()

,0,1,2,3,4,5
0,"[[[tensor([0, 0, 0], dtype=torch.uint8), tenso...","[tensor(1, dtype=torch.int16)]","[[tensor(1.1751), tensor(20.6786)]]",[[tensor(-0.1708)]],[[0]],[[0]]
1,"[[[tensor([0, 0, 0], dtype=torch.uint8), tenso...","[tensor(1, dtype=torch.int16)]","[[tensor(13.4320), tensor(41.2965)]]",[[tensor(-0.1705)]],[[0]],[[0]]
2,"[[[tensor([0, 0, 0], dtype=torch.uint8), tenso...","[tensor(1, dtype=torch.int16)]","[[tensor(18.2958), tensor(38.3567)]]",[[tensor(-0.1700)]],[[0]],[[0]]
3,"[[[tensor([0, 0, 0], dtype=torch.uint8), tenso...","[tensor(1, dtype=torch.int16)]","[[tensor(11.1235), tensor(51.8300)]]",[[tensor(-0.1694)]],[[0]],[[0]]
4,"[[[tensor([0, 0, 0], dtype=torch.uint8), tenso...","[tensor(1, dtype=torch.int16)]","[[tensor(-3.2229), tensor(44.3432)]]",[[tensor(-0.1706)]],[[0]],[[0]]


In [4]:
observations = df[0]

In [9]:
rgb = np.array([observation for observation in observations])

In [18]:
rgb.transpose(0, 1, 4, 3, 2).shape

(100000, 1, 3, 256, 256)

In [12]:
goal_indices = np.array([observation for observation in df[1]])

In [20]:
np.expand_dims(goal_indices, axis=1).shape

(100000, 1, 1)

In [13]:
goal_indices[0]

array([1], dtype=int16)

In [14]:
actions = np.array([observation for observation in df[2]])
rewards = np.array([observation for observation in df[3]])
dones = np.array([observation for observation in df[4]])
truncations = np.array([observation for observation in df[5]])

In [21]:
actions.shape, rewards.shape, dones.shape, truncations.shape

((100000, 1, 2), (100000, 1, 1), (100000, 1, 1), (100000, 1, 1))

In [7]:
obs_numpy = observations.to_numpy()

In [ ]:
obs_numpy

In [ ]:
# Extract 'rgb' from the first column (observations)
# Step 1: Extract and convert each 'rgb' (which is a list of lists of tensors) to NumPy arrays
rgb_images = df[0].apply(lambda obs: np.stack([t.numpy() for t in inner], axis=0) for inner in observations['rgb'])

# Step 2: Now each element in rgb_images is a 3D array — convert the list to a full NumPy array
rgb_images = np.array([np.stack(obs, axis=0) for obs in rgb_images])


# # Extract actions (assuming structure: [[tensor(x), tensor(y)]])
# actions = df[1].apply(lambda a: [t.item() for t in a[0]])

# # Extract rewards (assuming structure: [[tensor(x)]])
# rewards = df[2].apply(lambda r: r[0][0].item())

# # Extract dones (assuming structure: [[0]] or [[1]])
# dones = df[3].apply(lambda d: d[0][0])

# # Extract truncated flags (assuming structure: [[0]] or [[1]])
# truncated = df[4].apply(lambda t: t[0][0])

In [ ]:
np.array(rgb_images[0]).shape

(100000,)

In [ ]:
for experience in buffer.buffer:
    # Assuming each experience is a tuple of (state, action, reward, next_state, done)
    state, action, reward, next_state, done = experience

    # Print the state and action
    print("State:", state)
    print("Action:", action)
    print("Reward:", reward)
    print("Next State:", next_state)
    print("Done:", done)
    print()

In [ ]:
observations = np.array([experience[0] for experience in buffer.buffer])
actions = np.array([experience[1].cpu() for experience in buffer.buffer])
rewards = np.array([experience[2].cpu() for experience in buffer.buffer])
next_states = np.array([experience[3] for experience in buffer.buffer])
dones = np.array([experience[4].cpu() for experience in buffer.buffer])

In [ ]:
print("Observations shape:", observations.shape)
print("Actions shape:", actions.shape)
print("Rewards shape:", rewards.shape)
print("Next States shape:", next_states.shape)
print("Dones shape:", dones.shape)


In [ ]:

rgb_observations = np.array([obs['rgb'] for obs in observations])


In [ ]:
goal_index_obs = np.array([obs['goal_index'] for obs in observations])
goal_index_obs = np.expand_dims(goal_index_obs, axis=-1)

In [ ]:
rgb_observations.shape, goal_index_obs.shape

In [ ]:
for episode_id in range(len(data.observations["tasks"])):
    episode = data.observations["tasks"][episode_id]
    for step_id in range(len(episode)):
            steps = episode[step_id]
            if len(steps)>0:
                for task_id in range(len(steps)):
                    print(episode[step_id][task_id]["embedding"].shape)

In [ ]:
# red, green blue, yellow, white

color_index = {
    "red": 0,
    "green": 1,
    "blue": 2,
    "yellow": 3,
    "white": 4
}

colors = []
for episode_id in range(len(data.observations["tasks"])):
    episode = data.observations["tasks"][episode_id]
    for step_id in range(len(episode)):
            steps = episode[step_id]
            if len(steps)>0:
                for task_id in range(len(steps)):
                    colors.append(episode[step_id][task_id]["colour"])
set(colors)

In [ ]:
import numpy as np
embedding_size = 1152

color_index = {
    "red": 0,
    "green": 1,
    "blue": 2,
    "yellow": 3,
    "white": 4
}


all_embeddings = []
for batch_id in range(len(data.observations["tasks"])):
    batch_embeddings = []
    goal_batch = data.observations["goal_index"][batch_id]
    task_batch = data.observations["tasks"][batch_id]

    for image_id, task_list in task_batch.items():
        embedding = None
        if len(task_list) > 0:
            for task in task_list:
                if color_index.get(task.get("colour", None), None) == goal_batch[image_id][0]:
                    embedding = task.get("embedding", None)
        if embedding is None:
            embedding = np.zeros(embedding_size)
        
        batch_embeddings.append(embedding)

    batch_embeddings = np.array(batch_embeddings)
    all_embeddings.append(batch_embeddings)

all_embeddings = np.array(all_embeddings)
print(all_embeddings.shape)

In [ ]:
# Add the all_embeddings array to the observations
data.observations['embeddings'] = all_embeddings

In [ ]:
del data.observations['tasks']

In [ ]:
with open('/work/mech-ai-scratch/shreyang/me592/text2nav/embeddings/data/replay_buffer_with_embeddings.pkl', 'wb') as f:
    pickle.dump(data, f)

In [ ]:
import pickle

# Replace 'your_file.pkl' with the path to your pickle file
with open('/work/mech-ai-scratch/shreyang/me592/text2nav/embeddings/data/replay_buffer_with_embeddings.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
data.observations.keys()

In [ ]:
data.observations["embeddings"].shape

In [ ]:
total_nonzero_rows = 0

for embedding in data.observations["embeddings"]:
    arr = embedding
    nonzero_rows = np.sum(np.any(arr != 0, axis=1))
    total_nonzero_rows += nonzero_rows

print(total_nonzero_rows)